In [1]:
import eunomia
from eunomia.agents import Eunomia
from eunomia.tools import EunomiaTools

In [2]:
import logging
logging.getLogger().setLevel(logging.CRITICAL)
import warnings

# Disable all warning messages
warnings.filterwarnings("ignore")

## Obtaining Water Stability Information from a MOF Paper

In [3]:
paper_id = 69
paper_file =f'../data/{paper_id}.pdf'
docs_processor = eunomia.LoadDoc(file_name=paper_file, encoding="utf8")
sliced_pages = docs_processor.process(['references ', 'acknowledgement', 'acknowledgments', 'references\n'],
                                             chunk_size=1000, chunk_overlap=20, chunking_type='fixed-size')

In [4]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

Embedding_model = 'text-embedding-ada-002' 
faiss_index = FAISS.from_documents(sliced_pages, OpenAIEmbeddings(model=Embedding_model))

In [5]:
tools = EunomiaTools(tool_names = ['read_doc', 'eval_justification', 'recheck_justification'], vectorstore=faiss_index).get_tools()
agent = Eunomia(tools=tools, model='gpt-4', get_cost=True)
agent.run(prompt="Read the document and find the MOFS and their water stability.\
            \Check your answers and if the reasoning is not valid, try to find better reasons.")



> Entering new AgentExecutor chain...
I need to first read the document to identify the MOFs and their water stability.
Action: read_doc
Action Input: User's original promptThis model's maximum context length is 8192 tokens. However, your messages resulted in 9134 tokens. Please reduce the length of the messages.

k=9 results hitting the token limit for the chosen LLM. Reducing k and retrying...

This model's maximum context length is 8192 tokens. However, your messages resulted in 8459 tokens. Please reduce the length of the messages.

k=8 results hitting the token limit for the chosen LLM. Reducing k and retrying...


Observation: The document mentions several Metal-Organic Frameworks (MOFs) including UiO-66, MIL-140A, MIL-140B, MIL-140C, and MIL-140D. 

1. UiO-66: The water stability of UiO-66 is not explicitly mentioned in the document. However, it is mentioned that UiO-66 is stable up to 450°C under air and remains unaltered upon water adsorption/desorption cycles. This suggests

Action Input: "It appears that MIL-140s are highly stable with a slightly higher thermal stability, of around 500 8C under air."
Observation: The sentence does not provide information about the water stability of the found MOF. It only mentions the thermal stability of the MOF in air.
Thought:The justification for MIL-140D's water stability is not valid. I need to recheck the document for a better justification.
Action: recheck_justification
Action Input: MIL-140D
Observation: 1. The water stability of the MOF is Stable.
2. The probability score is 1.
3. The exact sentences from the document that justify my decision are: "First, it appears, based on XRPD analysis, that MIL-140 compounds are hydrothermally stable, that is, after an overnight dispersion in deionized water at 100 8C (Figure 3), no loss of crystallinity occurs, even for the upper analogues." and "Thus, it can be estimated that UiOs and their polymorph MIL-140s are the kinetic or thermodynamic phases, respectively. To our k

'The water stability of the MOFs is as follows: UiO-66 is stable, MIL-140A is stable, MIL-140B is stable, MIL-140C is stable, and MIL-140D is stable.'